<center>
<a href="http://www.insa-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo-insa.jpg" style="float:left; max-width: 120px; display: inline" alt="INSA"/></a> 

<a href="http://wikistat.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/wikistat.jpg" style="max-width: 250px; display: inline"  alt="Wikistat"/></a>

<a href="http://www.math.univ-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo_imt.jpg" style="float:right; max-width: 200px; display: inline" alt="IMT"/> </a>
</center>

 # [Apprentissage en Grande Dimension](https://github.com/wikistat/High-Dimensional-Learning ) : [Reconnaissance d'Activité Humaine](https://github.com/wikistat/High-Dimensional-Learning/tree/master/HumanActivityRecognition) ([*HAR*](https://archive.ics.uci.edu/ml/datasets/Human+Activity+Recognition+Using+Smartphones)) en <a href="https://www.python.org/"><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/f/f8/Python_logo_and_wordmark.svg/390px-Python_logo_and_wordmark.svg.png" style="max-width: 120px; display: inline" alt="Python"/></a>  Seconde partie:  apprentissage (profond) des signaux bruts  avec <a href="https://keras.io/"><img src="https://s3.amazonaws.com/keras.io/img/keras-logo-2018-large-1200.png" style="max-width: 100px; display: inline" alt="Keras"/></a>

Ce notebook présente la partie prediction de l'activité. Pour l'exploration, se référer au calepin afférent.

#   Introduction
##   Contexte
Les données sont issues de la communauté qui vise la reconnaissance d'activités humaines (*Human activity recognition, HAR*) à partir d’enregistrements, par exemple du gyroscope et de l'accéléromètre d'un smartphone.
Voir à ce propos l'[article](https://www.elen.ucl.ac.be/Proceedings/esann/esannpdf/es2013-11.pdf) relatant un colloque de 2013.  

Les données publiques disponibles ont été acquises, décrites et analysées par [Anguita et al. (2013)](https://www.elen.ucl.ac.be/Proceedings/esann/esannpdf/es2013-84.pdf). Elles sont accessibles sur le [dépôt](https://archive.ics.uci.edu/ml/datasets/Human+Activity+Recognition+Using+Smartphones) de l'University California Irvine (UCI) consacré à l'apprentissage machine ainsi que sur le site *Kaggle*.

L'archive contient les données brutes: accélérations en x, y, et z, chacun de 128 colonnes. D'autres fichiers en y soustrayant la gravité naturelle ainsi que les accélérations angulaires en x, y, et z, soit en tout 9 fichiers. Mais 6 utiles avec 6*128=768 mesures.

Les méthodes d'apprentissage sont appliquées sur ces données brutes, sans calculs préliminaires de caractéristiques (*features*).

##  Objectif
Cette deuxième étape s'intéresse aux données brutes. Est-il possible d'économiser le travail préliminaire de définition des variables métier en utilisant, par exemple, un algorithme d'apprentissage profond sur les données brutes ou leur transformation en ondelettes ?

**Objectif** Faire aussi bien (96% de bien classés) qu'avec les variables métier.

##  Méthodes abordées : 
**Attention l'accès à un environnement *GPU* est très vivement conseillé voire indispensable.**
- Modélisation, prévision de l'échantillon test par
   - Régression logistique (`Scikit-learn`) sur signaux "applatis"
   - Apprentissage profond en utilisant `Keras` 
       - MLP sur signaux "applatis"
       - MLP sur signaux mutlidimensionels
       - 1D Convolution
       - 2D Convolution
   - Validation Monte Carlo pour choisir le modèle
   
   
- ** Extensions possibles  à ce travail: **

    - Application des méthodes d'apprentissage classiques et de l'apprentissage profond sur les coefficients des décompositions des signaux en ondelettes
    - Optimisation des paramètres des différentes méthodes.
    - Améliorer l'architecture des réseaux ? 


#  Mise en place
## Librairies et initialisation

In [1]:
import pandas as pd
import numpy as np
import os
import time
import copy
import random
import itertools

#Utils Sklearn
import sklearn.linear_model as lm
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV,train_test_split

%matplotlib notebook
import matplotlib.pyplot as plt
import seaborn as sb
sb.set()

# DEEP LEARING
import tensorflow as tf
np.random.seed(42)
tf.set_random_seed(42)

# for reproducibility
# https://github.com/fchollet/keras/issues/2280
session_conf = tf.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)

from tensorflow.keras import backend as K
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

import tensorflow.keras.models as km 
import tensorflow.keras.layers as kl 

##  Prise en charge des données
### Sources

Les données sont celles originales du dépôt de l'[UCI](https://archive.ics.uci.edu/ml/datasets/Human+Activity+Recognition+Using+Smartphones). Elle peuvent être téléchargées en cliquant [ici](https://archive.ics.uci.edu/ml/machine-learning-databases/00240/UCI%20HAR%20Dataset.zip).

Elles contiennent deux jeux de dimensions différentes, chacun partagé en apprentissage et test.

1. Multidimensionel: un individus est constitué de 9 Séries Temporelles de *dimensions* $(N, 128, 9)$
2. Unidimensionnel: Les 9 Séries Temporelles sont concaténées pour constituer un vecteur de 128*9 = 1152 variables de *dimensions* $(N, 1152)*
        
Deux objets différents sont construits pour définir la variable $Y$ réponse car les librairies `Scikit-learn` et `Keras` prennent en compte des structures différentes: 
    
1. `Scikit-Learn`  Un vecteur de dimension $(N, 1)$ avec, pour chaque individu le numéro du label de l'activité de 0 à 5.
2. `Keras` Une matrice de dimension $(N, 6)$ des indicatrices (0 ou 1) des modalités de $Y$.

### Lecture des données

In [2]:
# attention: adapter si besoin le chemin d'accès aux données

DATADIR_UCI = '.'
SIGNALS = [ "body_acc_x", "body_acc_y", "body_acc_z", "body_gyro_x", "body_gyro_y", "body_gyro_z", "total_acc_x", "total_acc_y", "total_acc_z"]

def my_read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

def load_signal(data_dir, subset, signal):
    filename = f'{data_dir}/{subset}/Inertial Signals/{signal}_{subset}.txt'
    x = my_read_csv(filename).as_matrix()
    return x 

def load_signals(data_dir, subset, flatten = False):
    signals_data = []
    for signal in SIGNALS:
        signals_data.append(load_signal(data_dir, subset, signal)) 
    
    if flatten :
        X = np.hstack(signals_data)
    else:
        X = np.transpose(signals_data, (1, 2, 0))
        
    return X 

def load_y(data_dir, subset, dummies = False):
    filename = f'{data_dir}/{subset}/y_{subset}.txt'
    y = my_read_csv(filename)[0]
    
    
    if dummies:
        Y = pd.get_dummies(y).as_matrix()
    else:
        Y = y.as_matrix()
    
    return Y

Vérification des dimensions

In [3]:
#Multidimensional Data
X_train, X_test = load_signals(DATADIR_UCI, 'train'), load_signals(DATADIR_UCI, 'test')
# Flattened Data
X_train_flatten, X_test_flatten = load_signals(DATADIR_UCI, 'train', flatten=True), load_signals(DATADIR_UCI, 'test', flatten=True)

# Label Y
Y_train_label, Y_test_label = load_y(DATADIR_UCI, 'train', dummies = False), load_y(DATADIR_UCI, 'test', dummies = False)
#Dummies Y (For Keras)
Y_train_dummies, Y_test_dummies = load_y(DATADIR_UCI, 'train', dummies = True), load_y(DATADIR_UCI, 'test', dummies = True)

N_train = X_train.shape[0]
N_test = X_test.shape[0]

/anaconda3/envs/DL/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # This is added back by InteractiveShellApp.init_path()
/anaconda3/envs/DL/lib/python3.7/site-packages/ipykernel_launcher.py:34: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/anaconda3/envs/DL/lib/python3.7/site-packages/ipykernel_launcher.py:32: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [4]:
print("Dimension")
print("Données Multidimensionelles, : " + str(X_train.shape))
print("Données Unimensionelles, : " + str(X_train_flatten.shape))
print("Vecteur réponse (scikit-learn) : " + str(Y_train_label.shape))
print("Matrice réponse(Keras) : " + str(Y_train_dummies.shape))

Dimension
Données Multidimensionelles, : (7352, 128, 9)
Données Unimensionelles, : (7352, 1152)
Vecteur réponse (scikit-learn) : (7352,)
Matrice réponse(Keras) : (7352, 6)


#### Utilitaires

In [5]:
LABELS = ["WALKING","WALKING UPSTAIRS","WALKING DOWNSTAIRS","SITTING","STANDING","LAYING"]
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}


def my_confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

def _count_classes(y):
    return len(set([tuple(category) for category in y]))

# Apprentissage des signaux uni-dimensionels

La base d'apprentissage est de dimension (`N_train`, 1152)

##  Régression Logistique

La Régression Logistique est une des méthodes conduisant aux meilleurs résultats sur les variables métier.

In [6]:
t_start = time.time()
model_lr = lm.LogisticRegression(verbose=1)
model_lr.fit(X_train_flatten, Y_train_label)
t_end = time.time()
t_learning = t_end-t_start
score = model_lr.score(X_test_flatten, Y_test_label)
print("Score With Logistic Regression on Inertial Signals = %.2f, Learning time = %.2f secondes" %(score*100, t_learning) )
lr_prediction_label = model_lr.predict(X_test_flatten)
metadata_svc = {"time_learning" : t_learning, "score" : score}
pd.DataFrame(confusion_matrix(lr_prediction_label, Y_test_label), index = LABELS, columns=LABELS)

/anaconda3/envs/DL/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/envs/DL/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


[LibLinear]Score With Logistic Regression on Inertial Signals = 57.45, Learning time = 26.60 secondes


,WALKING,WALKING UPSTAIRS,WALKING DOWNSTAIRS,SITTING,STANDING,LAYING
WALKING,120,63,97,0,1,0
WALKING UPSTAIRS,74,218,56,23,72,27
WALKING DOWNSTAIRS,92,66,103,1,2,0
SITTING,79,32,58,397,112,0
STANDING,131,92,106,70,345,0
LAYING,0,0,0,0,0,510


**Q** Que dire de la performance?

## Perceptron multicouche

Un réseau de neurones classique est appris sur les données au même format que précédemment.

**Q** Expliciter les choix des paramètres et donc la structure du réseau.

In [7]:
epochs = 10
batch_size = 32
n_hidden = 32

n_features = X_train_flatten.shape[1]
n_classes=6


model_base_mlp_u =km.Sequential()
model_base_mlp_u.add(kl.Dense(n_hidden, input_shape=(n_features,),  activation = "relu"))
model_base_mlp_u.add(kl.Dropout(0.5))
model_base_mlp_u.add(kl.Dense(n_classes, activation='softmax'))
model_base_mlp_u.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

model_base_mlp_u.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                36896     
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 198       
Total params: 37,094
Trainable params: 37,094
Non-trainable params: 0
_________________________________________________________________


In [8]:
t_start = time.time()
model_base_mlp_u.fit(X_train_flatten,  Y_train_dummies, batch_size=batch_size, validation_data=(X_test_flatten, Y_test_dummies), epochs=epochs)
t_end = time.time()
t_learning = t_end-t_start

score = model_base_mlp_u.evaluate(X_test_flatten, Y_test_dummies)[1] 
print("Score With Simple MLP on Inertial Signals = %.2f, Learning time = %.2f secondes" %(score*100, t_learning) )
metadata_mlp_u = {"time_learning" : t_learning, "score" : score}
base_mlp_u_prediction = model_base_mlp_u.predict(X_test_flatten)

my_confusion_matrix(Y_test_dummies, base_mlp_u_prediction)

Train on 7352 samples, validate on 2947 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
7352/7352 [==============================] - 1s 81us/sample - loss: 1.1852 - acc: 0.5193 - val_loss: 0.9812 - val_acc: 0.6871
Epoch 2/10
7352/7352 [==============================] - 0s 57us/sample - loss: 0.8903 - acc: 0.6753 - val_loss: 0.8015 - val_acc: 0.7570
Epoch 3/10
7352/7352 [==============================] - 0s 58us/sample - loss: 0.7772 - acc: 0.7197 - val_loss: 0.7000 - val_acc: 0.8069
Epoch 4/10
7352/7352 [==============================] - 0s 61us/sample - loss: 0.7176 - acc: 0.7365 - val_loss: 0.6793 - val_acc: 0.8022
Epoch 5/10
7352/7352 [==============================] - 0s 57us/sample - loss: 0.6646 - acc: 0.7580 - val_loss: 0.6111 - val_acc: 0.8171
Epoch 6/10
7352/7352 [==============================] - 0s 57us/sample - loss: 0.6490 - acc: 0.7626 - val_loss: 0.5832 - val_acc: 0.8246
Epoch 7/10
7352/7352 [==============================] - 0s 57us/sample - loss: 0.6

Pred,LAYING,SITTING,STANDING,WALKING,WALKING_DOWNSTAIRS,WALKING_UPSTAIRS
True,,,,,,
LAYING,537,0,0,0,0,0
SITTING,0,331,139,0,3,18
STANDING,0,67,435,2,6,22
WALKING,0,19,1,416,50,10
WALKING_DOWNSTAIRS,0,3,6,14,357,40
WALKING_UPSTAIRS,0,0,0,24,68,379


** Q ** : Que conclure sur ces résultats en terme de performance, de temps d'apprentissage? Comparer avec la regression logistique?

** Exercice ** : Quelle est l'influence de l'ajout de nouvelle couche? Supression du Dropout? ...

# Apprentissage des signaux multidimensionnels
Les différents signaux ne sont pas concaténés en un seul signal mais pris en compte parallèlement.

## Perceptron multichouche
**Q** Expliciter les choix des paramètres et donc la structure du réseau.

In [9]:
n_hidden = 50

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = 6

model_base_mlp =km.Sequential()
model_base_mlp.add(kl.Dense(n_hidden, input_shape=(timesteps, input_dim),  activation = "relu"))
model_base_mlp.add(kl.Reshape((timesteps*n_hidden,) , input_shape= (timesteps, n_hidden)  ))
model_base_mlp.add(kl.Dense(n_classes, activation='softmax'))

model_base_mlp.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model_base_mlp.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 128, 50)           500       
_________________________________________________________________
reshape (Reshape)            (None, 6400)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 38406     
Total params: 38,906
Trainable params: 38,906
Non-trainable params: 0
_________________________________________________________________


In [10]:
t_start = time.time()
model_base_mlp.fit(X_train,  Y_train_dummies, batch_size=batch_size, validation_data=(X_test, Y_test_dummies), epochs=epochs)
t_end = time.time()
t_learning = t_end-t_start

score = model_base_mlp.evaluate(X_test, Y_test_dummies)[1] 
print("Score With Simple MLP on Multidimensional Inertial Signals = %.2f, Learning time = %.2f secondes" %(score*100, t_learning) )
metadata_mlp = {"time_learning" : t_learning, "score" : score}
base_mlp_prediction = model_base_mlp.predict(X_test)

my_confusion_matrix(Y_test_dummies, base_mlp_prediction)

Train on 7352 samples, validate on 2947 samples
Epoch 1/10
7352/7352 [==============================] - 1s 95us/sample - loss: 0.5988 - acc: 0.7542 - val_loss: 0.6925 - val_acc: 0.7265
Epoch 2/10
7352/7352 [==============================] - 1s 76us/sample - loss: 0.3434 - acc: 0.8671 - val_loss: 0.7386 - val_acc: 0.7302
Epoch 3/10
7352/7352 [==============================] - 1s 74us/sample - loss: 0.2581 - acc: 0.9021 - val_loss: 0.6058 - val_acc: 0.8120
Epoch 4/10
7352/7352 [==============================] - 1s 88us/sample - loss: 0.2088 - acc: 0.9248 - val_loss: 0.5268 - val_acc: 0.8510
Epoch 5/10
7352/7352 [==============================] - 1s 75us/sample - loss: 0.1766 - acc: 0.9290 - val_loss: 0.5328 - val_acc: 0.8385
Epoch 6/10
7352/7352 [==============================] - 1s 87us/sample - loss: 0.1531 - acc: 0.9397 - val_loss: 0.5169 - val_acc: 0.8524
Epoch 7/10
7352/7352 [==============================] - 1s 83us/sample - loss: 0.1359 - acc: 0.9460 - val_loss: 0.5148 - val_acc: 

Pred,LAYING,SITTING,STANDING,WALKING,WALKING_DOWNSTAIRS,WALKING_UPSTAIRS
True,,,,,,
LAYING,510,0,0,0,0,27
SITTING,0,333,133,1,0,24
STANDING,0,30,485,2,0,15
WALKING,0,0,0,422,68,6
WALKING_DOWNSTAIRS,0,0,0,25,395,0
WALKING_UPSTAIRS,0,0,0,81,19,371


## Réseau avec couche convolutionelle 1D (*ConvNet*)

L'idée pertinente avec ces données est évidemment d'identifier le problème lié au déphasage des signaux. L'utilisation d'une couche convolutionnelle introduit une propriété d'invariance par translation. Les caractéristiques ou *features* sortant de cette couche acquièrent donc ainsi de bonnes propriétés avant d'être dirigées vers des couches techniques intermédiaires (`MaxPooling, Flatten`) et une dernière couche de sortie qui effectue la discrimination à partir des caractéristiques.

**Q.** Remarquer le nombre de paramètres à estimer, le comparer avec celui du perceptron précédent et comprendre comment la convolution 1D agit sur les signaux (regarder en particulier la forme de la sortie de chaque couche du réseau). 

In [11]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = 6

#else:
model_base_conv_1D =km.Sequential()
model_base_conv_1D.add(kl.Conv1D(32, 9, activation='relu', input_shape=(timesteps, input_dim)))
model_base_conv_1D.add(kl.MaxPooling1D(pool_size=3))
model_base_conv_1D.add(kl.Flatten())
model_base_conv_1D.add(kl.Dense(n_classes, activation='softmax'))
model_base_conv_1D.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model_base_conv_1D.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 120, 32)           2624      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 40, 32)            0         
_________________________________________________________________
flatten (Flatten)            (None, 1280)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 7686      
Total params: 10,310
Trainable params: 10,310
Non-trainable params: 0
_________________________________________________________________


In [12]:
t_start = time.time()
model_base_conv_1D.fit(X_train,  Y_train_dummies, batch_size=batch_size, validation_data=(X_test, Y_test_dummies), epochs=epochs)
t_end = time.time()
t_learning = t_end-t_start

score = model_base_conv_1D.evaluate(X_test, Y_test_dummies)[1] 
print("Score With Conv on Multidimensional Inertial Signals = %.2f, Learning time = %.2f secondes" %(score*100, t_learning) )
metadata_conv = {"time_learning" : t_learning, "score" : score}
base_conv_1D_prediction = model_base_conv_1D.predict(X_test)

my_confusion_matrix(Y_test_dummies, base_conv_1D_prediction)

Train on 7352 samples, validate on 2947 samples
Epoch 1/10
7352/7352 [==============================] - 1s 157us/sample - loss: 0.5235 - acc: 0.8105 - val_loss: 0.4992 - val_acc: 0.8107
Epoch 2/10
7352/7352 [==============================] - 1s 128us/sample - loss: 0.1829 - acc: 0.9368 - val_loss: 0.3169 - val_acc: 0.8965
Epoch 3/10
7352/7352 [==============================] - 1s 127us/sample - loss: 0.1299 - acc: 0.9494 - val_loss: 0.2873 - val_acc: 0.9046
Epoch 4/10
7352/7352 [==============================] - 1s 129us/sample - loss: 0.1126 - acc: 0.9521 - val_loss: 0.2645 - val_acc: 0.9125
Epoch 5/10
7352/7352 [==============================] - 1s 127us/sample - loss: 0.1063 - acc: 0.9569 - val_loss: 0.2541 - val_acc: 0.9203
Epoch 6/10
7352/7352 [==============================] - 1s 121us/sample - loss: 0.1035 - acc: 0.9558 - val_loss: 0.2539 - val_acc: 0.9240
Epoch 7/10
7352/7352 [==============================] - 1s 127us/sample - loss: 0.0978 - acc: 0.9582 - val_loss: 0.2428 - va

Pred,LAYING,SITTING,STANDING,WALKING,WALKING_DOWNSTAIRS,WALKING_UPSTAIRS
True,,,,,,
LAYING,536,0,0,0,0,1
SITTING,4,369,111,0,0,7
STANDING,0,52,479,0,0,1
WALKING,0,0,0,489,6,1
WALKING_DOWNSTAIRS,0,0,0,3,412,5
WALKING_UPSTAIRS,0,0,0,2,21,448


## Réseau avec couche convolutionelle 2D (*ConvNet*)

**Q.** Remarquer le nombre de paramètres à estimer, le comparer avec celui du réseau précédent et comprendre comment la convolution 2D agit sur les signaux (regarder en particulier la forme de la sortie de chaque couche du réseau). 

In [13]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = 6

X_train_conv = X_train.reshape(N_train, timesteps, input_dim, 1)
X_test_conv = X_test.reshape(N_test, timesteps, input_dim, 1)

#else:
model_base_conv_2D =km.Sequential()
model_base_conv_2D.add(kl.Conv2D(32, (3, 9), activation='relu', input_shape=(timesteps, input_dim, 1)))
model_base_conv_2D.add(kl.MaxPooling2D(pool_size=(2, 1)))
model_base_conv_2D.add(kl.Flatten())
model_base_conv_2D.add(kl.Dense(n_classes, activation='softmax'))
model_base_conv_2D.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model_base_conv_2D.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 126, 1, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 63, 1, 32)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2016)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 6)                 12102     
Total params: 12,998
Trainable params: 12,998
Non-trainable params: 0
_________________________________________________________________


In [14]:
t_start = time.time()
model_base_conv_2D.fit(X_train_conv,  Y_train_dummies, batch_size=batch_size, validation_data=(X_test_conv, Y_test_dummies), epochs=epochs)
t_end = time.time()
t_learning = t_end-t_start

score = model_base_conv_2D.evaluate(X_test_conv, Y_test_dummies)[1] 
print("Score With Conv on Multidimensional Inertial Signals = %.2f, Learning time = %.2f secondes" %(score*100, t_learning) )
metadata_conv = {"time_learning" : t_learning, "score" : score}
base_conv_2D_prediction = model_base_conv_2D.predict(X_test_conv)

my_confusion_matrix(Y_test_dummies, base_conv_2D_prediction)

Train on 7352 samples, validate on 2947 samples
Epoch 1/10
7352/7352 [==============================] - 1s 156us/sample - loss: 0.6526 - acc: 0.7522 - val_loss: 0.6477 - val_acc: 0.7316
Epoch 2/10
7352/7352 [==============================] - 1s 131us/sample - loss: 0.3436 - acc: 0.8731 - val_loss: 0.5188 - val_acc: 0.8212
Epoch 3/10
7352/7352 [==============================] - 1s 130us/sample - loss: 0.2485 - acc: 0.9142 - val_loss: 0.4465 - val_acc: 0.8473
Epoch 4/10
7352/7352 [==============================] - 1s 128us/sample - loss: 0.1970 - acc: 0.9282 - val_loss: 0.4236 - val_acc: 0.8724
Epoch 5/10
7352/7352 [==============================] - 1s 127us/sample - loss: 0.1709 - acc: 0.9380 - val_loss: 0.3949 - val_acc: 0.8717
Epoch 6/10
7352/7352 [==============================] - 1s 139us/sample - loss: 0.1527 - acc: 0.9416 - val_loss: 0.3735 - val_acc: 0.8680
Epoch 7/10
7352/7352 [==============================] - 1s 128us/sample - loss: 0.1409 - acc: 0.9471 - val_loss: 0.3530 - va

Pred,LAYING,SITTING,STANDING,WALKING,WALKING_DOWNSTAIRS,WALKING_UPSTAIRS
True,,,,,,
LAYING,537,0,0,0,0,0
SITTING,0,375,106,0,0,10
STANDING,0,79,452,0,0,1
WALKING,0,0,0,404,75,17
WALKING_DOWNSTAIRS,0,0,0,8,407,5
WALKING_UPSTAIRS,0,0,0,3,44,424




**Attention au sur-apprentissage** A force de rechercher la meilleure architecture en minimisant l'erreur sur l'échantillon test, celle finalement trouvée peut y être très adaptée réduisant ainsi la capacité de généralisation. Il serait prudent de multiplier le découpage de l'échantillon par validation croisée *Monte Carlo*.

# Implémentation de la Validation Croisée Monte Carlo

**Objectif** :  trouver la meilleure architecture.

In [15]:
X=np.copy(np.concatenate((X_train, X_test), axis=0))
Y=np.copy(np.concatenate((Y_train_dummies, Y_test_dummies), axis=0))
Y.shape

(10299, 6)

In [16]:
epochs = 1
#Il faudrait en prendre plus ..
batch_size = 32
n_hidden = 32
n_classes = 6

In [17]:
N_MC=10

score=np.empty([N_MC,3])


for k in range(N_MC):
    print("\n \n *****************",k,"***************** \n")
    X_train_MC,X_test_MC,Y_train_dummies_MC,Y_test_dummies_MC=train_test_split(X,Y,test_size=0.2)
    N_train_MC = X_train_MC.shape[0]
    N_test_MC = X_test_MC.shape[0]
    
    timesteps = len(X_train_MC[0])
    input_dim = len(X_train_MC[0][0])
    
    X_train_conv_MC = X_train_MC.reshape(N_train_MC, timesteps, input_dim, 1)
    X_test_conv_MC = X_test_MC.reshape(N_test_MC, timesteps, input_dim, 1)
    
    
    # définition des modèles 
    model_base_mlp =km.Sequential()
    model_base_mlp.add(kl.Dense(n_hidden, input_shape=(timesteps, input_dim),  activation = "relu"))
    model_base_mlp.add(kl.Reshape((timesteps*n_hidden,) , input_shape= (timesteps, n_hidden)  ))
    model_base_mlp.add(kl.Dense(n_classes, activation='softmax'))
    model_base_mlp.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    
    model_base_conv_1D =km.Sequential()
    model_base_conv_1D.add(kl.Conv1D(32, 9, activation='relu', input_shape=(timesteps, input_dim)))
    model_base_conv_1D.add(kl.MaxPooling1D(pool_size=3))
    model_base_conv_1D.add(kl.Flatten())
    model_base_conv_1D.add(kl.Dense(n_classes, activation='softmax'))
    model_base_conv_1D.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    
    model_base_conv_2D =km.Sequential()
    model_base_conv_2D.add(kl.Conv2D(32, (3, 9), activation='relu', input_shape=(timesteps, input_dim, 1)))
    model_base_conv_2D.add(kl.MaxPooling2D(pool_size=(2, 1)))
    model_base_conv_2D.add(kl.Flatten())
    model_base_conv_2D.add(kl.Dense(n_classes, activation='softmax'))
    model_base_conv_2D.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    
   
    print("\n **** MLP **** \n")
    model_base_mlp.fit(X_train_MC,  Y_train_dummies_MC, batch_size=batch_size, validation_data=(X_test_MC, Y_test_dummies_MC), epochs=epochs)   

    print("\n **** conv 1D **** \n")
    model_base_conv_1D.fit(X_train_MC,  Y_train_dummies_MC, batch_size=batch_size, validation_data=(X_test_MC, Y_test_dummies_MC), epochs=epochs)
   
    print("\n **** conv 2D **** \n")
    model_base_conv_2D.fit(X_train_conv_MC,  Y_train_dummies_MC, batch_size=batch_size, validation_data=(X_test_conv_MC, Y_test_dummies_MC), epochs=epochs)
    
    score_mlp=model_base_mlp.evaluate(X_test_MC, Y_test_dummies_MC)[1]
    #score_lstm=model_base_lstm.evaluate(X_test_MC, Y_test_dummies_MC)[1]
    score_conv_1D=model_base_conv_1D.evaluate(X_test_MC, Y_test_dummies_MC)[1]
    score_conv_2D=model_base_conv_2D.evaluate(X_test_conv_MC, Y_test_dummies_MC)[1]
    s=[score_mlp,score_conv_1D,score_conv_2D]
    score[k,:]=s
    
final_scores=np.apply_along_axis(np.mean,0,score)*100
print("\n",score)
print(final_scores)


 
 ***************** 0 ***************** 


 **** MLP **** 

Train on 8239 samples, validate on 2060 samples
8239/8239 [==============================] - 1s 77us/sample - loss: 0.6300 - acc: 0.7418 - val_loss: 0.4933 - val_acc: 0.7961

 **** conv 1D **** 

Train on 8239 samples, validate on 2060 samples
8239/8239 [==============================] - 1s 125us/sample - loss: 0.5112 - acc: 0.8182 - val_loss: 0.2351 - val_acc: 0.9146

 **** conv 2D **** 

Train on 8239 samples, validate on 2060 samples
2060/2060 [==============================] - 0s 34us/sample - loss: 0.4067 - acc: 0.8573

 
 ***************** 1 ***************** 


 **** MLP **** 

Train on 8239 samples, validate on 2060 samples
8239/8239 [==============================] - 1s 86us/sample - loss: 0.6397 - acc: 0.7332 - val_loss: 0.4770 - val_acc: 0.8194

 **** conv 1D **** 

Train on 8239 samples, validate on 2060 samples
8239/8239 [==============================] - 1s 130us/sample - loss: 0.5721 - acc: 0.7810 - val_loss: 

In [18]:
print("MLP     :",round(final_scores[0],2),"% \n")
print("Conv 1D :",round(final_scores[1],2),"% \n")
print("Conv 2D :",round(final_scores[2],2),"% \n")

MLP     : 80.73 % 

Conv 1D : 89.59 % 

Conv 2D : 83.88 % 



**Q.** Quelle méthode est la plus performante ? Commenter ces résultats par rapport à ceux qui ont été obtenus sur les données "métier". 